In [1]:
import os,sys
from src.Mask.Utils import SaveModel,read_yaml,create_directory
from src.Mask.exceptions import CustomException
from src.Mask.loggers import logger
from src.Mask.Constants import *
from dataclasses import dataclass
from pathlib import Path

In [2]:
#step3) update the entity file: entity file is nothing but whaterver parameter we have define in yaml
#use those parameter to define class variable
@dataclass
class CallbackConfig():
    #defining the class variable
    root_dirpath:Path
    tensorboard_log_dirpath:Path
    model_checkpoint_path:Path
    all_param:dict

In [3]:
#step4)create configuration file in this file we read yaml file and create directories 
#and also assigning the value to class variable and taking rtn as function also
class ConfigurationManager():
    def __init__(self,config_filepath=CONFIG_FILEPATH,param_filepath=PARAM_FILEPATH):
        self.config = read_yaml(config_filepath) #rtn value as configbox dictatonary
        self.param = read_yaml(param_filepath)   #rtn value as configbox dictatonary

        #creating artifacts directory
        create_directory([self.config.artifact_root_dir]) #it ll create artifacts folder to project

    def get_call_model_config(self) ->CallbackConfig:
        #initializing the local variable
        config = self.config.prepare_callbacks
        param = self.param
        #getting dirname from this artifacts/callbacks/model_checkpoint/model.h5
        ckpt_directory = os.path.dirname(config.model_checkpoint_path) #create this path artifacts/callbacks/model_checkpoint

        #creating directory of callbacks
        create_directory([config.root_dirpath,ckpt_directory]) #create directoy artifacts/callbacks/model_checkpoint

        #creating an object of class variable and taking rtn as function
        callback_config = CallbackConfig(
            root_dirpath=config.root_dirpath,
            tensorboard_log_dirpath=config.tensorboard_log_dirpath,
            model_checkpoint_path=config.model_checkpoint_path,
            all_param=param
        )
        return callback_config

In [4]:
import tensorflow as tf
from datetime import datetime

In [5]:
#step5)updating the component file in this file we create object for class variable and perform task accordingly
class CallbackModel():
    def __init__(self,callbackconfig:CallbackConfig):
        self.callbackconfig = callbackconfig

    @property
    def _get_tensorboard_dir(self):
        timestamp = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
        filename = f"tensor_logfile{timestamp}.log"

        #if not exist this path make directory
        os.makedirs(self.callbackconfig.tensorboard_log_dirpath,exist_ok=True)

        return tf.keras.callbacks.TensorBoard(
            log_dir=os.path.join(self.callbackconfig.tensorboard_log_dirpath,filename),
              histogram_freq=1, write_graph=True, write_images=True
        )

    @property
    def _model_checkpoint(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.callbackconfig.model_checkpoint_path,
            monitor="val_loss",
            verbose = 1,
            save_best_only=True
        )

    @property
    def _early_stopping(self):
        return tf.keras.callbacks.EarlyStopping(
            monitor="val_loss",
            min_delta=self.callbackconfig.all_param.learning_rate,
            patience=2,
            verbose=1,
            mode="auto",
            restore_best_weights=True,
            start_from_epoch=0

        )
    
    #creating main method callback
    def callback(self):
        return [
            self._get_tensorboard_dir,
            self._model_checkpoint,
            self._early_stopping
        ]

In [6]:
os.chdir('../')
%pwd

'd:\\MaskImageDetection\\MaskDetectionModel'

In [7]:
#step6)updating the pipeline file
try:
    cm = ConfigurationManager()
    callback_config = cm.get_call_model_config()

    #creating an object of CallbackModel
    cbm = CallbackModel(callback_config)

    lst_obj = cbm.callback()
    for obj in lst_obj:
        logger.info(obj)

except Exception as e:
    raise CustomException(e,sys)

[2024-09-30 19:05:52,812]-INFO-33-Yaml file config\config.yaml reading
[2024-09-30 19:05:52,930]-INFO-33-Yaml file param.yaml reading
[2024-09-30 19:05:53,012]-INFO-49-Directory artifacts/callbacks created
[2024-09-30 19:05:53,014]-INFO-49-Directory artifacts/callbacks/model_checkpoint created
[2024-09-30 19:05:53,358]-INFO-11-<keras.src.callbacks.TensorBoard object at 0x000002051BF2C880>
[2024-09-30 19:05:53,359]-INFO-11-<keras.src.callbacks.ModelCheckpoint object at 0x000002051BF3E550>
[2024-09-30 19:05:53,360]-INFO-11-<keras.src.callbacks.EarlyStopping object at 0x000002051BF2C490>


In [8]:
%pwd

'd:\\MaskImageDetection\\MaskDetectionModel'